In [1]:
import json
import random


def generate_departments():
    departments = [
        {"id": 1, "name": "Software Engineering", "college": "Computing and Information Science"},
        {"id": 2, "name": "Computer Science", "college": "Computing and Information Science"},
        {"id": 3, "name": "Cybersecurity", "college": "Computing and Information Science"},
        {"id": 4, "name": "Biology", "college": "Natural Sciences"},
        {"id": 5, "name": "Microbiology", "college": "Natural Sciences"},
        {"id": 6, "name": "Science Laboratory Technology", "college": "Natural Sciences"},
        {"id": 7, "name": "Mathematics", "college": "Natural Sciences"},
        {"id": 8, "name": "Physics", "college": "Natural Sciences"},
        {"id": 9, "name": "Economics", "college": "Social Sciences"},
        {"id": 10, "name": "Accounting", "college": "Social Sciences"},
    ]
    return departments


def generate_venues():
    # Generate 29 venues with capacities
    return [
        {"id": i + 1, "name": f"Venue {i + 1}", "capacity": 50 + i * 10}
        for i in range(29)
    ]


def generate_courses():
    levels = [100, 200, 300, 400]
    courses = []
    departments = generate_departments()

    combined_courses = [
        {"course_code": "Math101", "course_name": "Basic Mathematics", "level": 100, "department_ids": [1, 2, 3]},
        {"course_code": "Math102", "course_name": "Advanced Mathematics", "level": 200, "department_ids": [4, 5, 6]},
        {"course_code": "GSP101", "course_name": "General Studies: Communication Skills", "level": 100,
         "department_ids": [1, 2, 3, 4, 5, 6]},
    ]

    for dept in departments:
        for level in levels:
            num_courses = 0

            # Determine number of courses based on level
            if level == 100:
                num_courses = random.randint(10, 14)
            elif level == 200:
                num_courses = random.randint(10, 14)
            elif level == 300:
                num_courses = random.randint(7, 10)
            elif level == 400:
                num_courses = random.randint(4, 6)

            # Generate departmental courses
            for i in range(1, num_courses + 1):
                courses.append({
                    "course_code": f"{dept['name'][:3].upper()}{level}{i:02d}",
                    "course_name": f"Course {i} for {dept['name']} Level {level}",
                    "department_id": dept["id"],
                    "level": level,
                    "students": random.randint(30, 200),  # Random student count
                })

    # Add combined courses
    for combined_course in combined_courses:
        combined_course["students"] = sum(random.randint(30, 150) for _ in combined_course["department_ids"])
        courses.append(combined_course)

    return courses


def assign_venues_to_courses(courses, venues):
    for course in courses:
        # Find the smallest venue that fits the number of students
        suitable_venues = [venue for venue in venues if venue["capacity"] >= course["students"]]
        if suitable_venues:
            # Assign the smallest suitable venue
            assigned_venue = min(suitable_venues, key=lambda x: x["capacity"])
            course["venue"] = assigned_venue["name"]
            course["invigilators"] = max(1, assigned_venue["capacity"] // 100)  # 1 invigilator per 100 students
        else:
            # No suitable venue found
            course["venue"] = "No suitable venue"
            course["invigilators"] = 0


def export_data():
    departments = generate_departments()
    venues = generate_venues()
    courses = generate_courses()

    # Assign venues to courses
    assign_venues_to_courses(courses, venues)

    data = {
        "departments": departments,
        "venues": venues,
        "courses": courses
    }
    with open("exam_data_with_venues.json", "w") as file:
        json.dump(data, file, indent=4)
    print("Data exported to exam_data_with_venues.json")


export_data()


Data exported to exam_data_with_venues.json


In [ ]:
import json
from datetime import datetime, timedelta

def load_data():
    with open("exam_data_with_venues.json", "r") as file:
        return json.load(file)


def generate_timetable(data):
    days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
    time_slots = ["8:00 AM - 10:00 AM", "10:30 AM - 12:30 PM", "1:00 PM - 3:00 PM", "3:30 PM - 5:30 PM"]
    
    timetable = []
    venue_usage = {venue["name"]: {"available": True, "day": None, "time_slot": None} for venue in data["venues"]}
    course_schedule = {}

    for course in data["courses"]:
        scheduled = False
        venue_name = course.get("venue")  # Retrieve the venue name

        # Check if the venue is valid and exists in the venue_usage dictionary
        if venue_name not in venue_usage:
            timetable.append({
                "course_code": course["course_code"],
                "course_name": course["course_name"],
                "department_id": course.get("department_id", "Combined"),
                "students": course["students"],
                "venue": "No suitable venue available",
                "invigilators": course["invigilators"],
                "day": "Unscheduled",
                "time_slot": "Unscheduled",
            })
            continue  # Skip to the next course

        for day in days:
            for time_slot in time_slots:
                # Check venue availability
                if venue_usage[venue_name]["available"] or (
                    venue_usage[venue_name]["day"] != day or venue_usage[venue_name]["time_slot"] != time_slot
                ):
                    # Schedule the course
                    timetable.append({
                        "course_code": course["course_code"],
                        "course_name": course["course_name"],
                        "department_id": course.get("department_id", "Combined"),
                        "students": course["students"],
                        "venue": venue_name,
                        "invigilators": course["invigilators"],
                        "day": day,
                        "time_slot": time_slot,
                    })

                    # Mark venue as used for this slot
                    venue_usage[venue_name] = {"available": False, "day": day, "time_slot": time_slot}
                    scheduled = True
                    break
            if scheduled:
                break

        if not scheduled:
            # If no slot found, mark as unscheduled
            timetable.append({
                "course_code": course["course_code"],
                "course_name": course["course_name"],
                "department_id": course.get("department_id", "Combined"),
                "students": course["students"],
                "venue": "No suitable venue available",
                "invigilators": course["invigilators"],
                "day": "Unscheduled",
                "time_slot": "Unscheduled",
            })

    return timetable



def assign_invigilators(timetable, data):
    invigilators = ["Invigilator " + str(i) for i in range(1, 31)]  # Example invigilators
    invigilator_schedule = {invigilator: [] for invigilator in invigilators}
    round_robin_index = 0

    for entry in timetable:
        if entry["invigilators"] > 0:
            assigned_invigilators = []
            for _ in range(entry["invigilators"]):
                invigilator = invigilators[round_robin_index % len(invigilators)]
                assigned_invigilators.append(invigilator)
                invigilator_schedule[invigilator].append({
                    "course_code": entry["course_code"],
                    "course_name": entry["course_name"],
                    "day": entry["day"],
                    "time_slot": entry["time_slot"],
                    "venue": entry["venue"]
                })
                round_robin_index += 1
            entry["assigned_invigilators"] = assigned_invigilators

    return invigilator_schedule


def export_timetable(timetable, invigilator_schedule):
    output = {
        "timetable": timetable,
        "invigilator_schedule": invigilator_schedule
    }
    with open("exam_timetable.json", "w") as file:
        json.dump(output, file, indent=4)
    print("Timetable and invigilator schedule exported to exam_timetable.json")


def main():
    data = load_data()
    timetable = generate_timetable(data)
    invigilator_schedule = assign_invigilators(timetable, data)
    export_timetable(timetable, invigilator_schedule)


main()


Timetable and invigilator schedule exported to exam_timetable.json


: 